In [281]:
# Various imports, setup
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [136]:
################
# Read in Data #
################
# For source see '../src/data_sources'

# NOAA; 1980 to today
noaa_quakes = pd.read_csv('../data/raw/noaa_quakes.tsv', sep='\t')
noaa_quakes = noaa_quakes.drop(columns = "Search Parameters", index = 0)
noaa_quakes

,Year,Location Name,Latitude,Longitude,Focal Depth (km),Mag,Deaths,Missing,Missing Description,Total Deaths,Total Missing,Total Missing Description
1,10.0,TURKMENISTAN: NISA,38.000,58.300,18.0,7.1,NaN,NaN,NaN,NaN,NaN,NaN
2,11.0,TURKEY,37.800,27.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17.0,"TURKEY: IZMIR, EFES, AYDIN, MANISA, ALASEHIR,...",37.850,27.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23.0,GREECE,38.200,22.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,25.0,PAKISTAN,33.000,72.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6175,2021.0,NEW ZEALAND: GISBORNE,-37.563,179.444,20.0,7.3,NaN,NaN,NaN,NaN,NaN,NaN
6176,2021.0,"KERMADEC ISLANDS: S OF, RAOUL",-29.613,-177.843,55.0,7.4,NaN,NaN,NaN,NaN,NaN,NaN
6177,2021.0,KERMADEC ISLANDS: SSE OF RAOUL ISLAND,-29.740,-177.267,19.0,8.1,NaN,NaN,NaN,NaN,NaN,NaN
6178,2021.0,ALGERIA: BEJAIA,36.915,5.199,8.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN


In [254]:
# Look at the available features

for col in noaa_quakes.columns:
    print(col)

Year
Location Name
Latitude
Longitude
Focal Depth (km)
Mag
Deaths
Missing
Missing Description
Total Deaths
Total Missing
Total Missing Description


## Convert to Log Scale for outcomes

In [255]:
noaa_quakes.shape

(6179, 12)

In [256]:
# Subset to deaths (outcome), Drop NA
quake_deaths = noaa_quakes[['Year', 'Mag', 'Total Deaths', 'Location Name']]

quake_deaths = quake_deaths[quake_deaths['Total Deaths'].notna()]


In [308]:
quake_deaths = quake_deaths.assign(Location=quake_deaths['Location Name'].str.split(':').str[0])

In [309]:
quake_deaths.shape

(1786, 8)

In [310]:
quake_deaths["Log Total Deaths"] = np.log(noaa_quakes["Total Deaths"])

In [311]:
quake_deaths.head()

,Year,Mag,Total Deaths,Location Name,Log Total Deaths,Magnitude Bin,Time Period,Location
20,79.0,NaN,2100.0,ITALY: NAPLES (NAPOLI),7.649693,10.0,Before 1900,ITALY
26,115.0,7.5,260000.0,TURKEY: ANTAKYA (ANTIOCH),12.468437,7.5,Before 1900,TURKEY
45,294.0,5.5,101.0,CHINA: BEIJING,4.615121,5.5,Before 1900,CHINA
54,342.0,NaN,40000.0,TURKEY: ANTAKYA (ANTIOCH),10.596635,10.0,Before 1900,TURKEY
57,344.0,NaN,101.0,CHINA: HENAN PROVINCE,4.615121,10.0,Before 1900,CHINA


## Bin Magnitudes

In [260]:
magbins = np.arange(start=0.5, stop=10, step=0.5)


quake_deaths['Magnitude Bin'] = (np.digitize(quake_deaths['Mag'], magbins, right=True)+1)/2

quake_deaths.head()

,Year,Mag,Total Deaths,Location Name,Log Total Deaths,Magnitude Bin
20,79.0,NaN,2100.0,ITALY: NAPLES (NAPOLI),7.649693,10.0
26,115.0,7.5,260000.0,TURKEY: ANTAKYA (ANTIOCH),12.468437,7.5
45,294.0,5.5,101.0,CHINA: BEIJING,4.615121,5.5
54,342.0,NaN,40000.0,TURKEY: ANTAKYA (ANTIOCH),10.596635,10.0
57,344.0,NaN,101.0,CHINA: HENAN PROVINCE,4.615121,10.0


In [261]:
quake_deaths['Magnitude Bin']

# deathbins = np.arange(start = 1000, stop = 830000, step = 1000)

# quake_deaths['Total Deaths Under (Bin)'] = (np.digitize(quake_deaths['Total Deaths'], deathbins, right=True))

# quake_deaths.head()

20      10.0
26       7.5
45       5.5
54      10.0
57      10.0
        ... 
6165     6.5
6166     6.5
6171     6.0
6173     5.5
6179     5.5
Name: Magnitude Bin, Length: 1786, dtype: float64

In [262]:
alt.data_transformers.enable('default', max_rows=None)


DataTransformerRegistry.enable('default')

In [263]:
alt.Chart(quake_deaths).mark_rect().encode(
    x='Year:N',
    y=alt.Y('Magnitude Bin:N', scale=alt.Scale(reverse=True)),
    color=alt.Color('Log Total Deaths:Q', scale=alt.Scale(scheme='reds'))
)

alt.Chart(...)

# Bin Era (Year)

In [264]:
time_period = []
for i in quake_deaths['Year']:
    if i < 1900:
        time_period.append("Before 1900")
    elif i >= 1900 and i < 1925:
        time_period.append("1900-1924")
    elif i >= 1925 and i < 1950:
        time_period.append("1925-1950")
    elif i >= 1950 and i < 1975:
        time_period.append("1950-1974")
    elif i >= 1975 and i < 2000:
        time_period.append("1975-1999")
    else:
        time_period.append("2000-2021")
        
quake_deaths["Time Period"] = time_period

In [265]:
# Before 1900 is at the end, ??FIX??
alt.Chart(quake_deaths).mark_rect().encode(
    x='Time Period:N',
    y=alt.Y('Magnitude Bin:N', scale=alt.Scale(reverse=True)),
    color=alt.Color('Log Total Deaths:Q', scale=alt.Scale(scheme='reds'))
)

alt.Chart(...)

# Group all years before 1900

In [321]:
df = quake_deaths.copy(deep=True)

years = []
for i in df["Year"]:
    if i < 1900:
        years.append("Before 1900")
    else:
        years.append(i)
        
df["Year (N)"] = years
df.head()

,Year,Mag,Total Deaths,Location Name,Log Total Deaths,Magnitude Bin,Time Period,Location,Year (N)
20,79.0,NaN,2100.0,ITALY: NAPLES (NAPOLI),7.649693,10.0,Before 1900,ITALY,Before 1900
26,115.0,7.5,260000.0,TURKEY: ANTAKYA (ANTIOCH),12.468437,7.5,Before 1900,TURKEY,Before 1900
45,294.0,5.5,101.0,CHINA: BEIJING,4.615121,5.5,Before 1900,CHINA,Before 1900
54,342.0,NaN,40000.0,TURKEY: ANTAKYA (ANTIOCH),10.596635,10.0,Before 1900,TURKEY,Before 1900
57,344.0,NaN,101.0,CHINA: HENAN PROVINCE,4.615121,10.0,Before 1900,CHINA,Before 1900


In [320]:
alt.Chart(df).mark_rect().encode(
    x='Year (N):N',
    y=alt.Y('Magnitude Bin:N', scale=alt.Scale(reverse=True)),
    color=alt.Color('Log Total Deaths:Q', scale=alt.Scale(scheme='reds'))
)

alt.Chart(...)

In [253]:
df.head()

,Year,Mag,Total Deaths,Location Name,Log Total Deaths,Magnitude Bin,Time Period
20,Before 1900,NaN,2100.0,ITALY: NAPLES (NAPOLI),7.649693,10.0,Before 1900
26,Before 1900,7.5,260000.0,TURKEY: ANTAKYA (ANTIOCH),12.468437,7.5,Before 1900
45,Before 1900,5.5,101.0,CHINA: BEIJING,4.615121,5.5,Before 1900
54,Before 1900,NaN,40000.0,TURKEY: ANTAKYA (ANTIOCH),10.596635,10.0,Before 1900
57,Before 1900,NaN,101.0,CHINA: HENAN PROVINCE,4.615121,10.0,Before 1900


In [328]:
df = df[df['Mag'].notna()]

magnitude = np.array(df['Mag']).reshape(-1,1)
log_deaths = np.array(df['Log Total Deaths'])

lm = LogisticRegression().fit(X = magnitude, y = log_deaths)

NameError: name 'LogisticRegression' is not defined

In [304]:
# Estimate of coefficient on Magnitude (slope)
lm.coef_

array([1.20855386])

In [306]:
# R-Squared
lm.score(magnitude, log_deaths)

0.1766467929720853

In [307]:
# Intercept
lm.intercept_

-4.498336310345763

In [327]:
chart = alt.Chart(quake_deaths).mark_circle(filled=True, 
                                size=200, 
                                opacity=0.9, 
                                color='tableau20').encode(
    x=alt.X('Mag', title='Magnitude'),
    y=alt.Y('Log Total Deaths', title='Log Total Deaths'),
    color=alt.Color('Location', legend=alt.Legend(title='Location')),
).configure_axis(
    tickCount=10
).properties(
    title='Earthquake Deaths by Location',
    width=600,
    height=400
)

chart

alt.Chart(...)

In [317]:
# ??FIX?? How to put regression line on scatter plot
# ??FIX?? How to transform magnitude into linear measure

chart + chart.transform_regression(x=alt.X('Mag', title='Magnitude'),
                                   y=alt.Y('Total Deaths', title='Total Deaths')).mark_line()

TypeError: transform_regression() got an unexpected keyword argument 'x'

In [331]:
df["Location"]

26          TURKEY
45           CHINA
67          GREECE
96           CHINA
101         TURKEY
           ...    
6165    BALKANS NW
6166     INDONESIA
6171      PAKISTAN
6173      COLOMBIA
6179         CHINA
Name: Location, Length: 1620, dtype: object